# Example from Section 7.1

This script verifies the example calculation from Section 7.1 of the paper.

In [1]:
%load_ext sage

In [2]:
lA, lB = 2, 3
eA, eB = 63, 41
f = 11
p = lA**eA * lB**eB * f - 1
assert p.is_prime()
k.<I> = GF(p)[]
K.<i> = GF(p^2, modulus=x^2+1)
E = EllipticCurve(K, [1,0])

In [3]:
PA = E(2374093068336250774107936421407893885897*i + 2524646701852396349308425328218203569693,
       1944869260414574206229153243510104781725*i + 1309099413211767078055232768460483417201)
PB = E(1556716033657530876728525059284431761206*i + 1747407329595165241335131647929866065215,
       3456956202852028835529419995475915388483*i + 1975912874247458572654720717155755005566)
QA = E(-PA[0], i*PA[1])
QB = E(-PB[0], i*PB[1])
assert (lA**eA)*PA == 0 and (lA**(eA-1))*PA != 0
assert (lB**eB)*PB == 0 and (lB**(eB-1))*PB != 0
assert PA.weil_pairing(QA, lA**eA).multiplicative_order() == lA**eA
assert PB.weil_pairing(QB, lB**eB).multiplicative_order() == lB**eB

In [4]:
mA, nA = 2575042839726612324, 8801426132580632841
mB, nB = 4558164392438856871, 20473135767366569910

In [5]:
def isogeny(E, l, e, R, P=None, Q=None):
    '''
    A very simple multiplication-based implementation of the SIDH isogeny evaluation procedure.
    '''
    if P is None:
        P = E(0)
    if Q is None:
        Q = E(0)
    x = E.base_field().polynomial_ring().gen()
    for i in reversed(range(e)):
        # Note: we could have written E.isogeny((l**i)*R), 
        # but Sage has a very slow path for this call.
        phi = E.isogeny(x - ((l**i)*R)[0])
        R, P, Q = phi(R), phi(P), phi(Q)
        E = phi.codomain()
    return E, P, Q

In [6]:
EA, phiPB, phiQB = isogeny(E, lA, eA, mA*PA + nA*QA, PB, QB)
EB, phiPA, phiQA = isogeny(E, lB, eB, mB*PB + nB*QB, PA, QA)

In [7]:
EA, phiPB, phiQB

(Elliptic Curve defined by y^2 = x^3 + (2091501916950999574452721858321602927843*i+972553154200337776765767083789464639451)*x + (60807325160986644487193630766046650344*i+3169253022785676431514890095429104981618) over Finite Field in i of size 3700444163740528325594401040305817124863^2,
 (2721153471195827394362378332379257637181*i + 1539185109368098978725926479650078750338 : 366199921913220734051920293872875427784*i + 1834684100381216170047241224726703679853 : 1),
 (2047412214418904136059667026856740906824*i + 210410807644625609951128565227377730537 : 39839443313971822568423776004766862687*i + 93242671882050419679778826538960350511 : 1))

In [8]:
EB, phiPA, phiQA

(Elliptic Curve defined by y^2 = x^3 + (613336589127545168753929747948723751445*i+1522892240764352004970989202275481325884)*x + (1751365643446878427298831224508754010925*i+514619434696964156988631199195423238937) over Finite Field in i of size 3700444163740528325594401040305817124863^2,
 (1840285667151684940152261485641077765166*i + 1205946128410465920445468025689008115483 : 3015693525350159684461695283215383069140*i + 1626164136473804366820814600761036680537 : 1),
 (1134287220707101080701212759958883254744*i + 2045900705277300793622003789178598520511 : 653462135239883274751711408898414405297*i + 33999787412505678391101800402560364101 : 1))

In [9]:
EAB, _, _ = isogeny(EB, lA, eA, mA*phiPA + nA*phiQA)
EBA, _, _ = isogeny(EA, lB, eB, mB*phiPB + nB*phiQB)

In [10]:
assert EAB.j_invariant() == EBA.j_invariant()
EAB.j_invariant()

1186454744475002430691715089880611825168*i + 519144330875457642568950846370688904962